In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import re
from rapidfuzz import process, fuzz
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from datetime import datetime

# Build engine
DATABASE_DSN = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:Czheyuan0227@"
    "aws-0-us-east-2.pooler.supabase.com:6543/postgres?sslmode=require"
)
engine = create_engine(DATABASE_DSN, pool_pre_ping=True)


#<---Item Listing Fuzyy Search--->
# Load CSV
Itemlisting = pd.read_csv("Item Listing.csv", dtype=str, encoding_errors="ignore").fillna("")
# Keep only the text after the first ":" in Item
Itemlisting["Item"] = Itemlisting["Item"].str.split(":", n=1).str[-1].str.strip()

Itemlisting.columns = Itemlisting.columns.str.strip()

def normalize(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9]", "", str(s)).lower()

# Normalized item column for searching
Itemlisting["__item_norm"] = Itemlisting["Item"].map(normalize)

def search_items(query: str, top_k: int = 15) -> pd.DataFrame:
    q = normalize(query)
    if not q:
        return pd.DataFrame(columns=Itemlisting.columns)

    # 1) Substring matches
    # Split query into words
    words = q.split()

    # Require all words to appear
    mask = Itemlisting["__item_norm"].apply(lambda name: all(w in name for w in words))
    direct = Itemlisting.loc[mask, Itemlisting.columns].copy()


    # 2) Fuzzy fallback 
    matches = process.extract(query, Itemlisting["Item"].tolist(), scorer=fuzz.WRatio, limit=top_k*2)
    fuzzy_rows = []
    seen = set()
    for _text, _score, idx in matches:
        if idx not in seen:
            seen.add(idx)
            fuzzy_rows.append(Itemlisting.iloc[idx])
    fuzzy = pd.DataFrame(fuzzy_rows)

    # 3) Combine results and dedupe by Item
    out = pd.concat([direct, fuzzy], ignore_index=True).drop_duplicates(subset=["Item"])
    return out.head(top_k)[["Item", "Description", 'Type', 'Cost', 'Price', 'Preferred Vendor']]


In [27]:
df.columns

Index(['Customer', 'Customer PO', 'WO', 'Product Number', 'Qty', 'Lead Time',
       'Picked', 'Part_Number', 'Reorder Pt (Min)', 'Max', 'On Hand',
       'On Sales Order', 'Stock_Available', 'Order', 'On PO', 'Reorder Qty',
       'Next Deliv', 'Sales/Week', 'Picked_Qty', 'ATP(LT)',
       'In Stock(Inventory)', 'Qty(+)', 'Available + installed PO',
       'Component_Status'],
      dtype='object')

In [26]:

# Read entire table
df = pd.read_sql_table("wo_structured", con=engine, schema="public")

ERP_df= df[['Order Date', 'Customer', 'Customer PO', 'WO', 'Product Number', 'Qty', 
                              "Available + installed PO", 'Stock_Available', 'ATP(LT)', 'In Stock(Inventory)', 'On Hand', 'On Sales Order', 'On PO',  'Lead Time', 'Picked']]

ERP_df = ERP_df.copy()
ERP_df['SO_Status'] = np.where(
    ERP_df['Stock_Available'] < 0, 'Shortage', 'Available'
)



ERP_df.loc[:, "Lead Time"] = pd.to_datetime(ERP_df["Lead Time"]).dt.date

filtered_df = ERP_df[ERP_df['WO'] == 'SO-20251402']

# filtered_df = ERP_df[ERP_df['Customer'] == 'Aerflite Canada Inc.']


# Apply styling
styled = (
    filtered_df.style
    .map(lambda v: 'background-color: red; color: white' if v == 'Shortage' else '', subset='SO_Status')
    .set_table_styles([
            {
                'selector': 'th.col_heading.level0.col' + str(filtered_df.columns.get_loc('Available + installed PO')),
                'props': [('background-color', 'orange'), ('color', 'black')]
            }
        ])
        .format(precision=0, na_rep='', thousands=",")
    )

# Display the styled DataFrame
display(styled)

KeyError: "['Order Date'] not in index"

In [7]:
filtered_df = ERP_df[ERP_df['Product Number'] == 'NRU-220S'].sort_values(by='Lead Time',ascending=True)
filtered_df

,Customer,Customer PO,WO,Product Number,Qty,Available + installed PO,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Lead Time,Picked,SO_Status
963,Aeva Inc.,PO15770,SO-20251402,NRU-220S,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-10-23,No,Available


## Part Name Lookup

In [8]:
results = search_items("cblm12")
print(results.to_string(index=False))

                           Item                                                                                         Description           Type  Cost Price           Preferred Vendor
         Cbl-M12A10M-DB9M-180CM Cbl-M12A10M-DB9M-180CM..M12 A-Code 10P (male) to DB9 (male), Length: 180CM. Industrial computer ... Inventory Part 14.00 25.00 Neousys Technology Incorp.
    Cbl-M12A17M-2DB9M_OW2-180CM Cbl-M12A17M-2DB9M_OW2-180CM..M12 (17-pole-A-coded) to 2xDB9 (Male) and 1xopen wire 2P, Length: 1... Inventory Part 15.00 19.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM1 Cbl-M12A17M-VGA-180CM1..M12 (17-pole-A-coded) to VGA (male), Length : 180CM. Industrial computer... Inventory Part 16.00 20.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM2                        Cbl-M12A17M-VGA-180CM2..M12 (17-pole-A-coded) to VGA (male), Length : 180CM. Inventory Part 10.00 13.00 Neousys Technology Incorp.
         Cbl-M12A17M-VGA-180CM3 Cbl-M12A17M-VGA-180CM3..M12 (17-pole-A

## Items that can still be assigned although have negative available

In [9]:
filtered_df1 = ERP_df[(ERP_df['Stock_Available'] < 0) & (ERP_df['Available + installed PO'] > 0)]
unique_parts_df = filtered_df1.drop_duplicates(subset=['Product Number'])
unique_parts_df[['Product Number', 'Stock_Available', 'Available + installed PO']]

,Product Number,Stock_Available,Available + installed PO
11,Ant-RP_SMAM-WiFi-196MM1,-237.0,13.0
77,M.2-LTE-7455,-94.0,31.0
78,Risr-M2B-mPCIe-SIMslot,-117.0,8.0
79,Cbl-MHF4-SMAF-30CM,-337.0,38.0
149,i9-13900,-4.0,10.0
151,M.280-SSD-2TB-PCIe44-TLC5WT-TD,-14.0,9.0
157,GC-Jetson-AGX32GB-Orin-Nvidia,-122.0,3.0
162,Cblkit-FP-NRU-230V-AWP_NRU-240S-AWP,-53.0,1.0
163,NRU-230V-AWP-JAO64G,-48.0,2.0
164,M.280-SSD-2TB-PCIe44-TLC5ET-TD,-34.0,17.0


## Ready to Pick (Ins Stock > Required Qty)

In [10]:
# True for WOs where ALL lines have In Stock > 0 and not been picked
wo_ok = (
    ERP_df.groupby("WO")
    .apply(lambda g: (g["In Stock(Inventory)"] >= g['Qty']).all() and (g["Picked"].eq("No").all()))
)


# 1) list of ready WOs
ready_wos = wo_ok[wo_ok].index.tolist()


# 2) full rows for those WOs
ready_df = ERP_df[ERP_df["WO"].isin(ready_wos)].copy()

print("Ready WOs:", ready_wos)


ready_df.to_excel(r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx", sheet_name="Sheet2", index=False)

Ready WOs: ['SO-20240315', 'SO-20250701', 'SO-20250726', 'SO-20250728', 'SO-20250936', 'SO-20251044', 'SO-20251111', 'SO-20251113', 'SO-20251126', 'SO-20251250', 'SO-20251280', 'SO-20251281', 'SO-20251283', 'SO-20251324', 'SO-20251338', 'SO-20251376', 'SO-20251377', 'SO-20251378', 'SO-20251391', 'SO-20251393', 'SO-20251394', 'SO-20251395', 'SO-20251419', 'SO-20251424']


/var/folders/5l/3_4dbl_x6t9cv8nl8tv33_800000gn/T/ipykernel_72071/3891689377.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["In Stock(Inventory)"] >= g['Qty']).all() and (g["Picked"].eq("No").all()))


## SO not has been assigned LT yet

In [25]:
# --- Your filter ---
assigned_mask = (
    (ERP_df["Lead Time"].dt.month.eq(7)  & ERP_df["Lead Time"].dt.day.eq(4)) |
    (ERP_df["Lead Time"].dt.month.eq(12) & ERP_df["Lead Time"].dt.day.eq(31))
)
Not_assgned_SO = ERP_df[assigned_mask]

# --- Save to Excel ---
output_path = r"Not_assigned_SO.xlsx"
Not_assgned_SO.to_excel(output_path, sheet_name="Sheet1", index=False)

# --- Open with openpyxl ---
wb = load_workbook(output_path)
ws = wb["Sheet1"]

# Find column indexes
wo_col = None
status_col = None
col_map = {}  # map header -> column index
for idx, cell in enumerate(ws[1], 1):  # header row
    col_map[cell.value] = idx
    if cell.value == "WO":
        wo_col = idx
    if cell.value == "SO_Status":
        status_col = idx

# Define fills
gray_fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
white_fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")

# Define red font + center alignment
red_font = Font(color="FF0000")
center_align = Alignment(horizontal="center", vertical="center")

# --- Apply formatting by WO groups ---
current_wo = None
fill_toggle = False

for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    wo_cell = row[wo_col - 1]
    status_cell = row[status_col - 1]

    # If WO changes, switch background
    if wo_cell.value != current_wo:
        current_wo = wo_cell.value
        fill_toggle = not fill_toggle

    # Pick background fill
    fill = gray_fill if fill_toggle else white_fill

    # Apply background fill to row
    for c in row:
        c.fill = fill

    # If shortage, make font red
    if status_cell.value == "Shortage":
        for c in row:
            c.font = red_font

# --- Set column widths ---
col_widths = {
    "Product Number": 30,
    "Customer": 30,
    "Customer PO": 30,
    "WO": 15,
    "Qty": 15,
    "Available + installed PO": 20,
    "Stock_Available": 15,
}

for col_name, width in col_widths.items():
    if col_name in col_map:
        col_letter = ws.cell(row=1, column=col_map[col_name]).column_letter
        ws.column_dimensions[col_letter].width = width

# --- Center align specific columns ---
for col_name in ["Qty", "Available + installed PO", "Stock_Available"]:
    if col_name in col_map:
        col_idx = col_map[col_name]
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
            for cell in row:
                cell.alignment = center_align

# --- Rename sheet to today's date ---
today_str = datetime.today().strftime("%Y-%m-%d")  # e.g., "2025-09-30"
ws.title = today_str

# Save
wb.save(output_path)

unique_wo_count = Not_assgned_SO["WO"].nunique()
print("# of unassigned WO:", unique_wo_count)

# of unassigned WO: 29


## Vendor POD

In [12]:
pod = df = pd.read_sql_table("Open_Purchase_Orders", con=engine, schema="public")
pod

,Order Date,QB Num,Name,Source Name,Item,Deliv Date,Qty(+)
0,2025/07/24,POD-251046,"CoastIPC, Inc.",Neousys Technology Incorp.,AccsyBx-6AntiVG-POC-551VTC,2025/10/03,34.0
1,2025/07/24,POD-251047,"CoastIPC, Inc.",Neousys Technology Incorp.,AccsyBx-6AntiVG-POC-551VTC,2025/11/21,24.0
2,2025/07/24,POD-251048,"CoastIPC, Inc.",Neousys Technology Incorp.,AccsyBx-6AntiVG-POC-551VTC,2025/12/19,29.0
3,2025/09/18,POD-251338,Neousys Technology Incorp.,Neousys Technology Incorp.,AccsyBx-Cardholder-9160GC-2000EAda,2025/10/01,1.0
4,2025/09/08,POD-251279,Neousys Technology Incorp.,Neousys Technology Incorp.,AccsyBx-FAN-NRU-100,2025/09/24,8.0
...,...,...,...,...,...,...,...
472,2025/09/09,POD-251293,Sim Sports Corporation,Neousys Technology Incorp.,Forwarding,2025/09/26,2.0
473,2025/04/21,POD-250574,Neousys Technology Incorp.,Neousys Technology Incorp.,,2025/09/09,1.0
474,2025/04/21,POD-250575,Neousys Technology Incorp.,Neousys Technology Incorp.,Certification,2025/09/09,1.0
475,2025/08/28,POD-251228,Neousys Technology Incorp.,Neousys Technology Incorp.,Engineer,2025/09/12,1.0


In [13]:
# Filter rows where Name is not 'Neousys Technology Incorp.'
filtered = pod[pod['Name'] != 'Neousys Technology Incorp.']

# Group by Item and sum Qty(+)
result = (
    filtered.groupby('Item', as_index=False)['Qty(+)']
    .sum()
)

lookup = (
    result[['Item', 'Qty(+)']]
    .drop_duplicates(subset=['Item'])         # ensures uniqueness
    .set_index('Item')['Qty(+)'] # Series: index = part_number
)

# 3) Map onto df (row count stays the same)
ERP_df['Qty(+)'] = ERP_df['Product Number'].map(lookup)
ERP_df



,Customer,Customer PO,WO,Product Number,Qty,Available + installed PO,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Lead Time,Picked,SO_Status,Qty(+)
0,"CoastIPC, Inc.",P96695,EO-20250002,RGS-8805GC,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-31,No,Available,NaN
1,Xanthon LLC,X110992,SO-20240315,POC-715,1.0,30.0,30.0,36.0,39.0,42.0,12.0,0.0,2025-12-19,No,Available,NaN
2,Xanthon LLC,X110992,SO-20240315,DDR5-16GB-WT48-IK,1.0,47.0,47.0,47.0,138.0,182.0,135.0,0.0,2025-12-19,No,Available,NaN
3,Xanthon LLC,X110992,SO-20240315,M.280-SSD-1TB-PCIe4-TLCWT-IK1,1.0,51.0,51.0,51.0,52.0,52.0,1.0,0.0,2025-12-19,No,Available,NaN
4,Xanthon LLC,X110992,SO-20240315,AccsyBx-FAN-POC-700,1.0,69.0,69.0,69.0,70.0,70.0,1.0,0.0,2025-12-19,No,Available,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,Noah Medical U.S.,PO209761,SO-20251423,Extnd-mPCIeHS,4.0,28.0,28.0,26.0,67.0,84.0,56.0,0.0,2025-12-31,No,Available,NaN
1042,Noah Medical U.S.,PO209761,SO-20251423,Cbl-MHF-RP_SMAF-30CM,8.0,26.0,26.0,22.0,92.0,118.0,92.0,25.0,2025-12-31,No,Available,NaN
1043,Noah Medical U.S.,PO209761,SO-20251423,Ant-RP_SMAM-WiFi-196MM1,8.0,13.0,-237.0,0.0,49.0,123.0,360.0,252.0,2025-12-31,No,Shortage,250.0
1044,Noah Medical U.S.,PO209761,SO-20251424,GC-RTXA4500-PNY,10.0,6.0,6.0,37.0,37.0,37.0,31.0,0.0,2025-12-31,No,Available,NaN


### Incoming QTY (assigned to WO) + Stock_Available <= 0

In [14]:
filtered_df1 = ERP_df[(ERP_df['Qty(+)']+ERP_df['Stock_Available']) <= 0].drop_duplicates(subset=['Product Number']).drop(['Customer', 'Customer PO', 'WO'], axis=1)
filtered_df1

,Product Number,Qty,Available + installed PO,Stock_Available,ATP(LT),In Stock(Inventory),On Hand,On Sales Order,On PO,Lead Time,Picked,SO_Status,Qty(+)
5,E-2278GE,5.0,-5.0,-6.0,0.0,19.0,29.0,35.0,6.0,2025-10-24,Picked,Shortage,1.0
98,M.280-SSD-1TB-PCIe44-TLC5-PN,10.0,-11.0,-24.0,0.0,29.0,39.0,63.0,43.0,2025-10-03,Picked,Shortage,13.0
99,Cbl-M12S4F-OW4-180CM1,10.0,0.0,-40.0,0.0,0.0,10.0,50.0,40.0,2025-10-03,Picked,Shortage,40.0
143,i7-9700E,20.0,-1.0,-21.0,0.0,-1.0,19.0,40.0,30.0,2025-10-01,Picked,Shortage,20.0
156,NRU-230V-AWP,2.0,0.0,-16.0,0.0,3.0,5.0,21.0,20.0,2025-08-06,Picked,Shortage,16.0
160,AC-IMX490-H120,1.0,0.0,-1.0,0.0,0.0,1.0,2.0,1.0,2025-08-06,Picked,Shortage,1.0
161,Cbl-M12A8M-2U2TA-180CM2,3.0,0.0,-4.0,0.0,0.0,3.0,7.0,4.0,2025-08-06,Picked,Shortage,4.0
165,SSD-2TB-TLC-IK,25.0,0.0,-50.0,0.0,25.0,25.0,75.0,50.0,2025-10-08,No,Shortage,50.0
167,Cblkit-BP-NRU-230V-AWP,25.0,0.0,-53.0,0.0,25.0,26.0,79.0,57.0,2025-10-08,No,Shortage,53.0
179,Ant-RP_SMAM-WiFi-108MM,8.0,0.0,-20.0,0.0,14.0,22.0,42.0,40.0,2025-09-11,Picked,Shortage,20.0
